# AntonovEtAl-2012-SSRN-SABR

* Antonov, A., & Spector, M. (2012). Advanced analytics for the SABR model. Available at SSRN. https://ssrn.com/abstract=2026350
* Antonov, A., Konikov, M., & Spector, M. (2013). SABR spreads its wings. Risk, 2013(Aug), 58–63.
* Antonov, A., Konikov, M., & Spector, M. (2019). Modern SABR Analytics. Springer International Publishing. https://doi.org/10.1007/978-3-030-10656-0

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import pandas as pd
import scipy as sp

### Uncomment below if you want to run on your modified code
#import sys
#sys.path.insert(sys.path.index('')+1, 'D:/Github/PyFENG')
import pyfeng as pf

In [3]:
# This is the list of parameters
sheets = list(range(1, 19))
pf.SabrHagan2002.init_benchmark() #.loc[sheets,]

,sigma,vov,rho,beta,texp,spot,col_name,Reference
Sheet,,,,,,,,
1,0.25,0.3000,-0.8000,0.3,10.00,1.00,IV MC,Table 1 in Antonov & Spector (2012). https://s...
2,0.25,0.3000,-0.8000,0.6,10.00,1.00,IV MC,Table 2 in Antonov & Spector (2012). https://s...
3,0.25,0.3000,-0.8000,0.9,10.00,1.00,IV MC,Table 3 in Antonov & Spector (2012). https://s...
4,0.25,0.3000,-0.5000,0.3,10.00,1.00,IV MC,Table 4 in Antonov & Spector (2012). https://s...
5,0.25,0.3000,-0.5000,0.6,10.00,1.00,IV MC,Table 5 in Antonov & Spector (2012). https://s...
6,0.25,0.3000,-0.5000,0.9,10.00,1.00,IV MC,Table 6 in Antonov & Spector (2012). https://s...
7,0.25,0.3000,-0.2000,0.3,10.00,1.00,IV MC,Table 7 in Antonov & Spector (2012). https://s...
8,0.25,0.3000,-0.2000,0.6,10.00,1.00,IV MC,Table 8 in Antonov & Spector (2012). https://s...
9,0.25,0.3000,-0.2000,0.9,10.00,1.00,IV MC,Table 9 in Antonov & Spector (2012). https://s...


In [4]:
# Select sheet number from above
sheet_no = 1
assert sheet_no in sheets
# Hagan2002 implementation
m, df, rv = pf.SabrHagan2002.init_benchmark(sheet_no)
print(df.iloc[:,[0, 2, 3]])
v_ref = df['IV Hagan'].values

      K  IV HL-P  IV Hagan
0   0.1   0.7898    0.7176
1   0.2   0.5992    0.5725
2   0.3   0.4999    0.4886
3   0.4   0.4339    0.4293
4   0.5   0.3849    0.3835
5   0.6   0.3461    0.3462
6   0.7   0.3141    0.3148
7   0.8   0.2869    0.2876
8   0.9   0.2634    0.2638
9   1.0   0.2427    0.2427
10  1.1   0.2244    0.2238
11  1.2   0.2080    0.2068
12  1.3   0.1936    0.1916
13  1.4   0.1808    0.1781
14  1.5   0.1697    0.1663
15  1.6   0.1602    0.1562
16  1.7   0.1523    0.1478
17  1.8   0.1460    0.1412
18  1.9   0.1411    0.1360
19  2.0   0.1373    0.1322


In [5]:
# Compare the reference values with the pyfeng implementation
v_pf = m.vol_for_price(**rv['args_pricing'])
np.testing.assert_almost_equal(v_ref, np.round(v_pf, 4))

In [6]:
m, df, rv = pf.SabrChoiWu2021P.init_benchmark(sheet_no)
m._base_beta = 1.0
v_ref = df['IV HL-P'].values
v_pf = m.vol_for_price(**rv['args_pricing'])
np.testing.assert_almost_equal(v_ref, np.round(v_pf, 4))